In [1]:
import os
from osgeo import gdal
import numpy as np
import rasterio
from scipy.spatial import ConvexHull

In [2]:
import rasterio
from rasterio.windows import Window
from rasterio.transform import rowcol

# AOI centroid coordinates
AOICentroid = (2042110.6, -3069083.8)

# Open the raster file
with rasterio.open("/home/tim/dentata/Sentinel2_seasonal/cvmsre_qld_m202403202405_abma2.vrt") as src:
    # Get the row and column of the centroid
    row, col = rowcol(src.transform, AOICentroid[0], AOICentroid[1])
    
    # Define the window size (100x100 pixels)
    window_size = 512
    
    # Calculate the window around the centroid
    window = Window(col - window_size // 2, row - window_size // 2, window_size, window_size)
    
    # Read the data within the window
    data = src.read(window=window)

In [ ]:
import numpy as np
from scipy.ndimage import convolve
from scipy.spatial import ConvexHull

# Define a 6x3x3 convolution kernel
kernel = np.ones((6, 3, 3))

# Initialize a list to store the convex hull volumes
convex_hullsVol = []

# Iterate over the 3D data to extract each 6x3x3 subset
for i in range(data.shape[1] - 2):
    for j in range(data.shape[2] - 2):
        subset = data[:, i:i+3, j:j+3]
        
        # Convolve the subset with the kernel
        convolved = convolve(subset, kernel, mode='constant', cval=0.0)
        
        # Calculate the convex hull for the convolved subset
        mask = convolved > 0
        points = np.argwhere(mask)
        if points.size > 0:
            print(points.shape)
            hull = ConvexHull(points)
            convex_hullsVol.append(hull.volume)
        else:
            convex_hullsVol.append(0)

# Make the convex hull volumes into a 2D array
convex_hullsVol = np.array(convex_hullsVol).reshape(data.shape[1] - 2, data.shape[2] - 2)

print(convex_hullsVol)

In [ ]:
data.shape

In [ ]:
# Plot the convex hull volumes
import matplotlib.pyplot as plt
plt.imshow(convex_hullsVol, cmap='viridis')

In [4]:
hull = ConvexHull(data)

In [ ]:
hull.volume

## Try a rios function using the segments to get the convex hull

In [3]:
def _getSegmentHull(info, inputs, outputs, otherargs):
    
    segments = inputs.segs[0]
    pca = inputs.PCA
    
    # Get unique segment IDs
    segIDs = np.unique(segments)
            
    # Get the PCA values for each segment
    for segID in segIDs:
        if segID > 0:
            mask = segments == segID
            pca_values = pca[:,mask]
            pca_values = pca_values.T
            # Calculate the convex hull volume
            try:
                hull = ConvexHull(pca_values)
                hullVol = hull.volume
                
                otherargs.hullVols[segID] = hullVol
            except Exception as e:
                print(e)
                otherargs.hullVols[segID] = 9999
            
        else:
            otherargs.hullVols[segID] = 9999

In [23]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

nbar=np.array([[16111,44864,33636], [17049,45301,34168],[17704,46421,33194],[16260,45225,33001]])

# Standardize the data
scaler = StandardScaler()
nbar = scaler.fit_transform(nbar)
print(nbar)

print(nbar.shape)

hull = ConvexHull(nbar) 
hullvol = hull.volume
print(hullvol)

[[-1.0450606  -1.01005717  0.30327301]
 [ 0.41802424 -0.2603417   1.48742893]
 [ 1.43968797  1.66112586 -0.68055577]
 [-0.8126516  -0.390727   -1.11014617]]
(4, 3)
0.1806144834228752


In [2]:
from rios import applier, cuiprogress
from scipy.spatial import ConvexHull
import numpy as np
from sklearn.preprocessing import StandardScaler

def _getSegmentHull(info, inputs, outputs, otherargs):
    segments = inputs.segs[0]
    pca = inputs.PCA
    
    # Get unique segment IDs
    segIDs = np.unique(segments)
                
    # Get the PCA values for each segment
    for segID in segIDs:
        if segID > 0:
            mask = segments == segID
            pca_values = pca[:, mask]
            pca_values = pca_values.T
            pca_length = pca_values.shape[0]            
            if pca_length >=4:
            
                pca_values = StandardScaler().fit_transform(pca_values)
            
                
                # Calculate the convex hull volume
                try:
                    hull = ConvexHull(pca_values)
                    hullVol = hull.volume
                    # print(f"Hull Volume for Segment {segID}: {hullVol}")
                    
                    # Check if there's already a value in the dictionary
                    existing_vol, existing_length = otherargs.hullVols.get(segID, (0, 0))
                    # print(f"Existing Volume: {existing_vol}, Existing Length: {existing_length}")
                    
                    # Compare lengths and update dictionary
                    if pca_length > existing_length:
                        # print(f"Updating Segment {segID}: New Length {pca_length}, Old Length {existing_length}")
                        otherargs.hullVols[segID] = (hullVol, pca_length)
                    else:
                        otherargs.hullVols[segID] = (existing_vol, existing_length)
                except Exception as e:
                    print(f"Segment ID: {segID}, PCA Length: {pca_length}")
                    print(f"Error calculating hull for Segment {segID}: {e}")
                    # If there's an error, keep the existing value or set to 65535 if not present
                    existing_vol, existing_length = otherargs.hullVols.get(segID, (0, 0))
                    if existing_length == 0:
                        otherargs.hullVols[segID] = (65535, existing_length)
            

        else:
            existing_vol, existing_length = otherargs.hullVols.get(segID, (0, 0))
            otherargs.hullVols[segID] = (max(existing_vol, 0), 0)
            

def getSegmentHull():    
    # Create the RIOS file objects
    infiles = applier.FilenameAssociations()
    outfiles = applier.FilenameAssociations()

    # Setup the IO
    infiles.segs = "/home/tim/rubella/scripts/SpectralBotany/data/Sentinel/Sentinel_brigalow_PCA_v2_segs_id.tif"
    infiles.PCA = "/home/tim/rubella/scripts/SpectralBotany/data/Sentinel/Sentinel_brigalow_PCA_v2.tif"

    # Get the otherargs
    otherargs = applier.OtherInputs()
    otherargs.hullVols = {}

    # Controls for the processing   
    controls = applier.ApplierControls()
    controls.windowxsize = 1024
    controls.windowysize = 1024
    controls.setStatsIgnore(0) #  nodata
    controls.progress = cuiprogress.CUIProgressBar()
    controls.setFootprintType("INTERSECTION")
    controls.setResampleMethod("near")
    controls.setOutputDriverName("GTIFF")
    controls.setCreationOptions(["COMPRESS=DEFLATE",
                                    "ZLEVEL=9",
                                    "PREDICTOR=2",
                                    "BIGTIFF=YES",
                                    "TILED=YES",
                                    "INTERLEAVE=BAND",
                                    "NUM_THREADS=ALL_CPUS",
                                    "BLOCKXSIZE=512",
                                    "BLOCKYSIZE=512"])
    controls.setOverlap = 512

    # # Set concurrency depending on system
    # conc = applier.ConcurrencyStyle(numReadWorkers=3,
    #                                 numComputeWorkers=2,
    #                                 computeWorkerKind="CW_THREADS",
    #                                 )

    # controls.setConcurrencyStyle(conc)

    # Run the function
    applier.apply(_getSegmentHull, infiles, outfiles, otherargs, controls=controls)
    
    return otherargs.hullVols

HullVols = getSegmentHull()   

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2424


KeyboardInterrupt: 

In [7]:
# Export the hull volumes to a raster
from rios import applier, cuiprogress

def _writeSegmentHull(info, inputs, outputs, otherargs):
    
    segments = inputs.segs[0]
    nodata = segments == 0
    
    outputrast = np.zeros(segments.shape, dtype=np.float32) 
       
    # Use the dictionary to write the hull volumes to the output raster
    for segID in otherargs.hullVols.keys():
        mask = segments == segID
        outputrast[mask] = otherargs.hullVols[segID][0] # Get volumes not lengths
    
    outputNodat = outputrast == 65535
    #rescale to uint16
    outputrast = (outputrast/otherargs.maxVol)*65534 
    # Mask no data
    outputrast[nodata] = 65535
    outputrast[outputNodat] = 65535
    
    outputrast = outputrast.reshape(1, outputrast.shape[0], outputrast.shape[1])
    
    outputs.hullRast = outputrast.astype(np.uint16)




# Get the no data value
# ds = gdal.Open("/home/tim/dentata/Sentinel2_seasonal/cvmsre_qld_m202109202111_abma2.vrt")
# noData = ds.GetRasterBand(1).GetNoDataValue()

def writeSegmentHull():    
    # Create the RIOS file objects
    infiles = applier.FilenameAssociations()
    outfiles = applier.FilenameAssociations()

    # Setup the IO
    infiles.segs = "/home/tim/rubella/scripts/SpectralBotany/data/Landsat/LandsatBarest_brigalow_PCA_250m_segs_id.tif"

    outfiles.hullRast = "/home/tim/rubella/scripts/SpectralBotany/data/Landsat/LandsatBarest_brigalow_PCA_segs_hullVol16_v5.tif"

    # Get the otherargs
    otherargs = applier.OtherInputs()
    otherargs.hullVols = HullVols
    values = list(HullVols.values())
    volumes = [x[0] for x in values]
    otherargs.minVol = min(volumes)
    otherargs.maxVol = max(volumes)

    # Controls for the processing   
    controls = applier.ApplierControls()
    controls.windowxsize = 512
    controls.windowysize = 512
    controls.setStatsIgnore(65535) #  nodata
    controls.progress = cuiprogress.CUIProgressBar()
    controls.setFootprintType("INTERSECTION")
    controls.setResampleMethod("near")
    controls.setOutputDriverName("GTIFF")
    controls.setCreationOptions(["COMPRESS=DEFLATE",
                                    "ZLEVEL=9",
                                    "PREDICTOR=2",
                                    "BIGTIFF=YES",
                                    "TILED=YES",
                                    "INTERLEAVE=BAND",
                                    "NUM_THREADS=ALL_CPUS",
                                    "BLOCKXSIZE=512",
                                    "BLOCKYSIZE=512"])

    # Set concurrency depending on system
    # conc = applier.ConcurrencyStyle(numReadWorkers=3,
    #                                 numComputeWorkers=2,
    #                                 computeWorkerKind="CW_THREADS",
    #                                 )

    # controls.setConcurrencyStyle(conc)

    # Run the function
    print("Processing PCA")
    applier.apply(_writeSegmentHull, infiles, outfiles, otherargs, controls=controls)

writeSegmentHull()

Processing PCA

Computing Pyramid Layers...

Computing Statistics...


In [18]:
# get range of values in HullVols
values = np.array(list(HullVols.values()))
values = values[values != 65535]
print(np.min(values), np.max(values))
scaled = (values - np.min(values)) / (np.max(values) - np.min(values)) *65534
scaled = scaled.astype(np.uint16)

0.0 530.0


In [ ]:
print(np.min(scaled), np.max(scaled))

In [4]:
# search hulvolls values for 65535

for key, value in HullVols.items():
    if value[0] == 65535:
        print(key, value)


## Function for Rao's Q

In [15]:
nums, counts = HullVols.get(50000000, (0, 0))

In [16]:
counts

0